In [8]:
! pip install \
  autogen-agentchat \
  autogen-core~=0.4.5 \
  autogen-ext==0.4.5 \
  "autogen-ext[azure]" \
  azure-ai-inference~=1.0.0b8 \
  azure-ai-projects \
  azure-search-documents>=11.5.2 \
  chainlit \
  chromadb~=0.6.3 \
  ipykernel~=6.29.5 \
  "mcp[cli]" \
  mistralai~=0.4.2 \
  openai \
  pillow==11.0.0 \
  pydantic \
  python-dotenv~=1.0.1 \
  "semantic-kernel>=1.27.2" \
  "semantic-kernel[azure]>=1.27.2" \
  "semantic-kernel[mcp]>=1.27.2"

In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML
from tmdbv3api import TMDb, Genre
from openai import AsyncOpenAI
from tmdbv3api import TMDb
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function
from tmdbv3api import TMDb, Discover
from tmdbv3api import Movie
import pandas as pd
from datetime import datetime
from tmdbv3api import TMDb, Discover
import os, json
from tmdbv3api import TMDb, Genre, Discover
import semantic_kernel as sk

In [ ]:
tmdb = TMDb()
tmdb.api_key="2fa30f6a1d22eb80c6dc9cac9cc67bdc"

In [ ]:
movie = Movie()

recommendations = movie.recommendations(movie_id=111)

for recommendation in recommendations:
    print(recommendation.title)
    print(recommendation.overview)

The Untouchables
Elliot Ness, an ambitious prohibition agent, is determined to take down Al Capone. In order to achieve this goal, he forms a group given the nickname “The Untouchables”.
Carlito's Way
A Puerto-Rican ex-con, just released from prison, pledges to stay away from drugs and violence despite the pressure around him, and lead a better life outside NYC.
Reservoir Dogs
A botched robbery indicates a police informant, and the pressure mounts in the aftermath at a warehouse. Crime begets violence as the survivors -- veteran Mr. White, newcomer Mr. Orange, psychopathic parolee Mr. Blonde, bickering weasel Mr. Pink and Nice Guy Eddie -- unravel.
Heat
Obsessive master thief Neil McCauley leads a top-notch crew on various daring heists throughout Los Angeles while determined detective Vincent Hanna pursues him without rest. Each man recognizes and respects the ability and the dedication of the other even though they are aware their cat-and-mouse game may end in violence.
GoodFellas
Th

In [ ]:
# Setup TMDb with your API key
tmdb = TMDb()
tmdb.api_key = '2fa30f6a1d22eb80c6dc9cac9cc67bdc'
# Create a Discover instance
discover = Discover()

# Get top drama movies sorted by popularity
# Genre ID 18 is for Drama
drama_movies = discover.discover_movies({
    'with_genres': 18,
    'sort_by': 'popularity.desc',
    'page': 1
})

drama_movies

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/k32XKMjmXMGeydykD32jfER3BVI.jpg', 'genre_ids': [28, 9648, 18], 'id': 1045938, 'original_language': 'en', 'original_title': 'G20', 'overview': 'After the G20 Summit is overtaken by terrorists, President Danielle Sutton must bring all her statecraft and military experience to defend her family and her fellow leaders.', 'popularity': 288.9191, 'poster_path': '/wv6oWAleCJZUk5htrGg413t3GCy.jpg', 'release_date': '2025-04-09', 'title': 'G20', 'video': False, 'vote_average': 6.651, 'vote_count': 361}, {'adult': False, 'backdrop_path': '/1YMrOtrW7b4pL2lfD8UciZPOJGs.jpg', 'genre_ids': [18, 53, 9648], 'id': 974576, 'original_language': 'en', 'original_title': 'Conclave', 'overview': 'After the unexpected death of the Pope, Cardinal Lawrence is tasked with managing the covert and ancient ritual of electing a new one. Sequestered in the Vatican with the Catholic Church’s most powerful leaders until the process is complete, Lawrence finds h

In [ ]:
def explore_features(api_key, pages=5, sort_by="popularity.desc"):
    """
    Fetch the first `pages` pages of TMDb discover results and compute
    min/max for popularity, vote_average, vote_count, and release_year.
    """
    tmdb = TMDb()
    tmdb.api_key = api_key
    discover = Discover()

    records = []
    for page in range(1, pages + 1):
        params = {
            "sort_by": sort_by,
            "page": page
        }
        movies = discover.discover_movies(params)
        for m in movies:
            # parse release year
            year = None
            if m.release_date:
                try:
                    year = datetime.strptime(m.release_date, "%Y-%m-%d").year
                except ValueError:
                    pass
            records.append({
                "popularity": m.popularity,
                "vote_average": m.vote_average,
                "vote_count": m.vote_count,
                "release_year": year
            })

    df = pd.DataFrame(records)
    stats = df.agg({
        "popularity": ["min", "max"],
        "vote_average": ["min", "max"],
        "vote_count": ["min", "max"],
        "release_year": ["min", "max"]
    })
    return stats


In [5]:
tmdb.api_key = 'TMDB_API_KEY'
print(explore_features('2fa30f6a1d22eb80c6dc9cac9cc67bdc', pages=5, sort_by="popularity.desc"))

     popularity  vote_average  vote_count  release_year
min     39.5407         3.400           2          1966
max    942.1209         8.489       37002          2025


In [6]:
df = explore_features('2fa30f6a1d22eb80c6dc9cac9cc67bdc', pages=5, sort_by="popularity.desc")
print(df.head())

     popularity  vote_average  vote_count  release_year
min     39.5407         3.400           2          1966
max    942.1209         8.489       37002          2025


In [7]:
#Max / Min of popularity, vote_average, vote_count, release_date
movies = discover.discover_movies({'page': 1})
pops   = [m.popularity    for m in movies]
votes  = [m.vote_average  for m in movies]
counts = [m.vote_count    for m in movies]
dates  = [m.release_date  for m in movies]

print("Popularity:",   min(pops), "to", max(pops))
print("Vote Avg:",     min(votes), "to", max(votes))
print("Vote Count:",   min(counts), "to", max(counts))
print("Release Date:", min(dates), "to", max(dates))


Popularity: 130.0792 to 942.1209
Vote Avg: 5.286 to 7.732
Vote Count: 15 to 4816
Release Date: 2004-02-25 to 2025-04-23


In [9]:
#Number of APi calls per page 
movies = discover.discover_movies({'page': 1})
print("Results per call:", len(movies))


Results per call: 20


In [10]:
discover.discover_movies({
    'with_genres': '18,10749',  # Drama + Romance
    'sort_by': 'popularity.desc',
    'page': 1
})

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/qiVrapzgrDO7c6yQTSvn0dhwsn8.jpg', 'genre_ids': [10749, 18], 'id': 1252309, 'original_language': 'es', 'original_title': 'Pídeme lo que quieras', 'overview': "After his father's death, Eric Zimmerman travels to Spain to oversee his company's branches. In Madrid, he falls for Judith and engage in an intense, erotic relationship full of exploration. However, Eric fears his secret may end their affair.", 'popularity': 110.6035, 'poster_path': '/k82T5L1utX1G2LhNWeza9nGUKPk.jpg', 'release_date': '2024-11-29', 'title': 'Ask Me What You Want', 'video': False, 'vote_average': 5.9, 'vote_count': 150}, {'adult': False, 'backdrop_path': '/oz4U9eA6ilYf1tyiVuGmkftdLac.jpg', 'genre_ids': [10749, 18], 'id': 1010581, 'original_language': 'es', 'original_title': 'Culpa mía', 'overview': "Noah must leave her city, boyfriend, and friends to move into William Leister's mansion, the flashy and wealthy husband of her mother Rafaela. As a proud and i

In [42]:
def recommend_movies(
    genre_id: int,
    preference: str = "newer",
    top_n: int    = 5,
    weight_pop: float = 0.5,
    weight_vote: float = 0.5
):
    """
    Fetch page 1 for `genre_id`, score by weighted(popularity, vote_average),
    filter by age (≤5 yrs new vs >5 yrs old), return top_n dicts.
    """
    current_year = datetime.now().year
    raw = discover.discover_movies({
        "with_genres": genre_id,
        "sort_by":     "popularity.desc",
        "page":        1
    })
    scored = []
    for m in raw:
        try:
            year = int(m.release_date[:4])
        except:
            continue
        age = current_year - year
        if preference == "newer" and age > 5:  continue
        if preference == "older" and age <= 5: continue
        score = weight_pop * m.popularity + weight_vote * m.vote_average
        scored.append({"title": m.title, "year": year, "score": score})
    return sorted(scored, key=lambda x: x["score"], reverse=True)[:top_n]


In [43]:
class MoviePlugin:
    @kernel_function(description="Recommend movies by genre, date preference, and number of results")
    def recommend(self, genre_id: str, preference: str = "newer", top_n: str = "5") -> str:
        recs = recommend_movies(
            genre_id=int(genre_id),
            preference=preference,
            top_n=int(top_n)
        )
        lines = [f"{i+1}. {m['title']} ({m['year']}) — score={m['score']:.2f}"
                 for i, m in enumerate(recs)]
        return "\n".join(lines)


In [13]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
print("Endpoint:", os.getenv("AZURE_OPENAI_ENDPOINT"))


Endpoint: https://<your-resource>.openai.azure.com/


In [14]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

In [15]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
#Defining MoviePlugin Skill
class MoviePlugin:
    @kernel_function(description="Recommend movies by genre, date preference, and number of results")
    def recommend(self, genre_id: str, preference: str = "newer", top_n: str = "5") -> str:
        recs = recommendations(
            genre_id=int(genre_id),
            preference=preference,
            top_n=int(top_n)
        )
        return "\n".join(
            f"{i+1}. {m['title']} ({m['year']}) — score={m['score']:.2f}"
            for i, m in enumerate(recs)
        )

In [ ]:

# 1) Initialize TMDb client
tmdb = TMDb()
tmdb.api_key ='2fa30f6a1d22eb80c6dc9cac9cc67bdc'

# 2) Fetches the official list of movie genres
genre_client = Genre()
all_genres = genre_client.movie_list()  

# 3) Build a lookup dict
GENRE_MAP = {g.name.lower(): g.id for g in all_genres}
print("Official TMDb genres:")
for name, gid in GENRE_MAP.items():
    print(f"  {name:15s} → {gid}")


Official TMDb genres:
  action          → 28
  adventure       → 12
  animation       → 16
  comedy          → 35
  crime           → 80
  documentary     → 99
  drama           → 18
  family          → 10751
  fantasy         → 14
  history         → 36
  horror          → 27
  music           → 10402
  mystery         → 9648
  romance         → 10749
  science fiction → 878
  tv movie        → 10770
  thriller        → 53
  war             → 10752
  western         → 37


In [ ]:
# OpenAI async client
from openai import AsyncOpenAI

# Load your .env (TMDB_API_KEY, GITHUB_TOKEN, AZURE_OPENAI_*)
load_dotenv()


True

In [20]:
# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key ='2fa30f6a1d22eb80c6dc9cac9cc67bdc'
# Fetch the official list of movie genres
genre_client = Genre()
all_genres = genre_client.movie_list()

# Build a name→ID lookup (lowercased keys)
GENRE_MAP = {g.name.lower(): g.id for g in all_genres}

# Create a Discover instance for querying movies
discover = Discover()

# (print to verify
print("Loaded genres:", list(GENRE_MAP.keys())[:5], "…")


Loaded genres: ['action', 'adventure', 'animation', 'comedy', 'crime'] …


In [23]:
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)


In [34]:

agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[MoviePlugin()],
    name="MovieAgent",
    instructions="You will identify the preferred genres that the customer wants based on their prompt",
)

In [37]:
class MoviePlugin:
    @kernel_function(
        description="Recommend movies by genre (name or ID), date preference, and number of results"
    )
    def recommend(
        self,
        genre: str,
        preference: str = "newer",
        top_n: str    = "5"
    ) -> str:
        if genre.isdigit():
            gid = int(genre)
        else:
            gid = GENRE_MAP.get(genre.lower())
            if gid is None:
                return (
                    f"Unknown genre '{genre}'. "
                    f"Available: {', '.join(GENRE_MAP.keys())}"
                )
        try:
            n = int(top_n)
        except:
            n = 5

        recs = recommend_movies(genre_id=gid, preference=preference, top_n=n)

        return "\n".join(
            f"{i+1}. {m['title']} ({m['year']}) — score={m['score']:.2f}"
            for i, m in enumerate(recs)
        )


In [ ]:
user_inputs = [
    "I like comedy movies recommend me some.",
    "I don't like that one, please show me horror.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

In [ ]:
#Important note to reader the way the score is calculated is the following : 
# The “score” you’re seeing under the hood is just a simple weighted combination of each movie’s popularity and its vote_average.
# I put the weights as equal importance; 
# However we can tweak this later but so far, based on rating seen online, it seems to be following the correct logic.